In [1]:
import numpy as np
from optimizers import OptimizationModel
def run_optimization():
    # Define random bounds for the variables
    bounds = [
        {'name': 'var1', 'type': 'continuous', 'domain': (0, 50)},
        {'name': 'var2', 'type': 'continuous', 'domain': (0, 50)},
        {'name': 'var3', 'type': 'continuous', 'domain': (0, 50)}
    ]

    # Define a random target value
    target_value = np.random.randint(1, 200)
    print(target_value)

    # Define random reagent information
    reagent_info = {'reagent1': 20, 'reagent2': 30, 'reagent3': 25, 'reagent4': 50, 'reagent5': 15}  # Assuming volumes in μL

    # Define fixed reagents (randomly chosen)
    fixed_reagents = [('reagent2', 30), ('reagent3', 25), ('reagent4', 50)]  # Example fixed reagents with volumes

    # Initialize the optimization model
    opt_model = OptimizationModel(bounds, target_value, reagent_info, fixed_reagents, initial_design_numdata=15, batch_size=3, max_iters=40)

    def simulate_chemistry(x):
        # Define coefficients to simulate more nuanced chemistry behavior
        a, b, c = 1.0, 0.1, 0.05
        
        # Simulate an outcome with more predictable behavior
        result = a * x[:, 0] + b * x[:, 1]**2 - c * x[:, 2]
        
        # Add a small, controlled amount of noise to mimic experimental variation
        #noise = np.random.normal(0, 1, size=x.shape[0])
        return (result)
    def calc_obj(x):
        return abs(target_value - x).reshape(-1, 1)

    # Generate initial design and simulate experiments to get initial data
    X_initial = opt_model.generate_initial_design()
    Y_initial = calc_obj(simulate_chemistry(X_initial))

    print(type(X_initial))
    print(type(Y_initial))

    #print("x init", X_initial, 'yinit',Y_initial.shape)
    opt_model.experiment_data['X'] = list(X_initial)
    opt_model.experiment_data['Y'] = list(Y_initial)
    # Initialize the optimizer with initial experimental data
    opt_model.initialize_optimizer(X_initial, Y_initial)

    print(opt_model.optimizer)
    print(opt_model.acquisition)

    # Learning loop
    number_of_iterations = 75  # Define the number of iterations for the learning loop
    for i in range(number_of_iterations):
        # Suggest next locations for experiments
        next_locations = opt_model.suggest_next_locations()
        print(next_locations)
        
        # Simulate experiments at the suggested locations
        Y_new = calc_obj(simulate_chemistry(next_locations))
        
        # Update the optimizer with the new experimental data
        opt_model.update_experiment_data(next_locations, Y_new)
        
        print(f"Iteration {i+1}, Suggested Locations: {next_locations}, Results: {Y_new}")
        print(opt_model.quit, opt_model.curr_iter)
        
        # Optional: Check if the optimizer has met the target or other stopping criteria
        if opt_model.quit:
            print("Stopping criteria met.")
            break
    # To get the best observed X values (parameters)
    X_best = opt_model.optimizer.X[np.argmin(opt_model.optimizer.Y)]

    opt_model.optimizer

    # To get the best observed Y value (function value)
    Y_best = np.min(opt_model.optimizer.Y)
    return i, target_value, (X_best, Y_best)


In [2]:
test_info = {}
for test in range(2):
    num_iter, tar, best = run_optimization()
    test_info[test] = (num_iter, tar, best)


45
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
1
MPI
[[25.12537648 14.45398169 42.53698107]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Iteration 1, Suggested Locations: [[25.12537648 14.45398169 42.53698107]], Results: [[1.1097139]]
False 1
2
LCB
[[17.76591439  9.17897839 32.95952767]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Iteration 2, Suggested Locations: [[17.76591439  9.17897839 32.95952767]], Results: [[20.45669757]]
False 2
0
EI
[[50.  0.  0.]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Iteration 3, Suggested Locations: [[50.  0.  0.]], Results: [[5.]]
False 3
1
MPI
[[37.95272924  0.         50.        ]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Iteration 4, Suggested Locations: [[37.95272924  0.         50.        ]], Results: [[9.54727076]]
False 4
2
LCB
[[35.26493258  0.          0.        ]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Iteration 5, Suggested Locations: [[35.26493258  0.          0.        ]], Results: [[9.73506742]]
False 5
0

In [3]:
vals = test_info.values()
vals

dict_values([(38, 45, (array([25.        , 15.        , 41.66666667]), 0.4166666666666643)), (39, 104, (array([34.32642012, 27.0218713 , 31.79472267]), 1.7548368266340333))])

In [4]:
# These are the total number of iters before reaching some stopping criteria for each test
h = [val[0] for val in vals]
# These are the target values used in each test
i = [val[1] for val in vals]
print(h)
print(i)

# Print average nuber of iterations to meet stopping criteria with 3 variable parameters and max iters of 40:
print(sum(h)/len(h))

[38, 39]
[45, 104]
38.5
